## 5.2 Essential Functionality | 핵심기능

* Series나 DataFrame에 저장된 데이터를 다루는 기본적 방법

### 5.2.1 Reindexing | 재색인 

In [40]:
import pandas as pd
import numpy as np
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c']) #- 색인 직접 지정
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [41]:
#- 새로운 색인에 맞도록 객체 재생성. 즉 색인-색인값 쌍을 유지하면서 인덱스 재설정
#- 존재하지 않는 색인값이 있다면 NaN 추가

obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e']) #- reindex(인덱스 리스트)
obj2


a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [42]:
#- method 옵션으로 시계열 같은 순차적 데이터를 재색인할 때, 값을 보간하거나 값 채우기 

obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [50]:
obj3.reindex(range(6), method='nearest') #- backfill 우선 -> ffil ?_?

0      blue
1    purple
2    purple
3    yellow
4    yellow
5    yellow
dtype: object

* DataFrame.<span style="color:#F8766D">reindex</span>(labels=None, index=None, columns=None, axis=None, method=None, copy=True, level=None, fill_value=nan, limit=None, tolerance=None)  

    method : {None, ‘backfill’/’bfill’, ‘pad’/’ffill’, ‘nearest’}
    Method to use for **filling holes** in reindexed DataFrame.  
    Please note: <span style="color:#F8766D">**this is only applicable to DataFrames/Series with a monotonically increasing/decreasing index.**</span>  

    None (default): don’t fill gaps  
    pad / ffill: Propagate last valid observation forward to next valid.  
    backfill / bfill: Use next valid observation to fill gap.  
    nearest: Use nearest valid observations to fill gap.  

    참고 : [reindex](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html)  

In [52]:
#- DataFrame에 대한 reindex는 로우(색인), 컬럼 또는 둘 다 변경 가능
#- 순서만 전달하면 로우가 재색인됨.

frame = pd.DataFrame(np.arange(9).reshape((3, 3)), #- arange 는 ndarray 반환
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])

frame
#frame['Ohio']

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [45]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2
#frame2['Ohio']

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


* 비어있는 ['b']를 인덱스로 추가하면서, float인 NaN 값에 맞춰 나머지 컬럼의 값도 정수->실수로 변환 

In [46]:
states = ['Texas', 'Utah', 'California']
frame2.reindex(columns=states) #- columns 예약어를 사용하여 재색인

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


**예약어란?**  
컴퓨터 프로그래밍 언어에서 이미 문법적인 용도로 사용되고 있기 때문에 식별자로 사용할 수 없는 단어

In [47]:
#- loc을 이용해 라벨로 색인하면 더욱 간결함
#- 보통 이러한 명시적 방법 선호

states = ['Texas', 'Utah', 'California']
frame.loc[['a', 'b', 'c', 'd'], states]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['b'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [53]:
states = ['Texas', 'California']
frame.loc[['a', 'c', 'd'], states]

,Texas,California
a,1,2
c,4,5
d,7,8


KeyError: "Passing list-likes to **.loc or [] with any missing labels is no longer supported.**   
The following labels were missing: **Index(['b'], dtype='object').**   
See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"    

Index(['b']), column Utah의 값이 없는데, 이처럼 missing label이 있을 경우, .loc을 통한 재색인 불가능 

### 5.2.2 Dropping Entries from an Axis | 하나의 로우나 컬럼 삭제하기 
* 색인 배열, 또는 삭제하려는 로우나 컬럼이 제외된 리스트를 이미 가지고 있다면, 로우나 컬럼을 쉽게 삭제할 수 있음.  
* 이를 위해서는 **데이터의 모양을 변경**하는 작업이 필요함.  
* drop 함수처럼 Series나 DataFrame의 크기 또는 형태를 변경하는 함수는 새로운 객체를 반환하는 대신 원본 객체를 변경함.

In [55]:
#- drop 메서드를 사용하면 선택한 값들이 삭제된 새로운 객체를 얻을 수 있음. 
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj
new_obj = obj.drop('c')
new_obj
#obj.drop(['d', 'c']) 

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [56]:
obj.drop('c') #- 원본 객체를 변경하는것
obj #- 원본은 그대로 있음

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [58]:
#- inplace 옵션은 버려지는 값 모두 삭제 (주의!)
#- 두 번 누르면 발생하는 KeyError: "['c'] not found in axis"
#- inplace=True의 경우, '변경한 원본객체'가 '오리지널 원본객체'를 inplace 한다고 볼 수 있음. 

obj.drop('c', inplace=True) 
obj

KeyError: "['c'] not found in axis"

In [59]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [60]:
#- 해당 로우(axis0) 값 모두 삭제

data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [61]:
#- 컬럼의 값 삭제 시 axis=1 또는 axis='columns'

data.drop('two', axis=1) 
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### 5.2.3 Indexing, Selection, and Filtering | 색인하기, 선택하기, 거르기
* Series의 색인(obj[...]은 NumPy 배열의 색인과 유사하게 동작하지만, **정수가 아니어도 된다는 점이 다름**  << 위치 정수(integer position) 또는 라벨
* 4.1.4 NumPy의 색인 참고 -> 교재 144pg~155pg 

In [62]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd']) # [0,1,2,3]
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [69]:
obj['b'] #- 색인 ['b']에 대한 값
obj[1] #- 색인 [1]번째 값
obj[2:4] #- 색인 [2]번째 부터 [4]번째 직전까지의 값 [2,4)
obj[['b', 'a', 'd']]
obj[[1, 3]] #- 색인 [1]번째와 [3]번째 값
obj[obj < 2] #- [obj<2]를 만족하는 로우 불러오기

a    0.0
b    1.0
dtype: float64

In [71]:
#- 라벨이름으로 슬라이싱하면, 시작점과 끝점을 포함함 (기억!)

obj['b':'c'] 
obj[2:3] 

c    2.0
dtype: float64

In [72]:
#- 선택된 영역에 값 대입
obj['b':'c'] = 5 
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [75]:
#- 색인으로 DataFrame에서 하나 이상의 컬럼 값을 가져올 수 있음

data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data
data['two']
data[['three', 'one']] #- 컬럼 리스트

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [76]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [80]:
#- 슬라이싱으로 로우를 선택하거나 불리언 배열로 로우를 선택할 수 있음

data[:2] #- [0,2) 시작점 포함, 끝점 미포함
data[data['three'] > 5] #- []안의 조건을 만족하는 로우를 선택
data[data['three']==10]

,one,two,three,four
Utah,8,9,10,11


In [84]:
#- 스칼라 비교를 이용해 생성된 불리언 DataFrame 이용

data < 5 #- Ohio 로우전체와 (Colorado,one) 만 True
data[data < 5] = 0 #- data<5 가 True인 값만 0으로 변환
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selection with loc and iloc | loc 과 iloc으로 선택하기
* DataFrame의 로우에 대해 라벨로 색인하는 방법으로서, 특수한 색인 필드인 loc과 iloc   
* NumPy와 비슷한 방식에 추가적으로 축의 라벨을 사용하여 DataFrame의 로우와 컬럼을 선택할 수 있음.   
* 축 이름을 선택할 때는 loc을, 정수 색인으로 선택할 때는 iloc을 사용  

* Different choices for indexing
    * **.loc** is primarily label based, but may also be used with a boolean array.
    * **.iloc** is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array. 
    [loc과 iloc](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)

In [85]:
#- loc을 이용하여 축의 "라벨"로 하나의 로우와 여러 컬럼을 선택하기

data.loc['Colorado', ['two', 'three']] #- 'Colorado' 로우 1개, 'two','three' 컬럼 2개 

two      5
three    6
Name: Colorado, dtype: int64

In [88]:
#- iloc을 통해 "정수" 색인으로도 위와 비슷하게 선택할 수 있음

data.iloc[2, [3, 0, 1]] #- [2]번째 로우인 Utah, [3,0,1] 순서대로 'four', 'one', 'two' 컬럼 3개
data.iloc[2] #- [2]번쨰 로우 전체 값 선택
data.iloc[[1, 2], [3, 0, 1]] 

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [92]:
#- loc과 iloc 함수는 슬라이스, 단일 라벨, 라벨 리스트 모두 지원

data.loc[:'Utah', 'two']
data.iloc[:, :3]
#data.iloc[:, :3][data.three > 5] #- [data.three > 5] 조건성취 못하는 Ohio 배제

,one,two,three
Ohio,0,0,0
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


### Integer Indexes | 정수 색인
* 정수 색인으로 pandas 객체를 다루다보면 리스트나 튜플 같은 파이썬 내장 자료구조에서 색인을 다루는 방법과의 차이점 때문에 실수하게 되는 경우가 있음. 
* 아래 예시에서, pandas는 라벨 색인을 찾는 데 실패하므로 정수 색인으로 값을 찾음. (탐색 순서 : 라벨 -> 정수 색인)
* 라벨 색인이 0,1,2를 포함하는 경우, 사용자가 라벨 색인으로 선택하려는 것인지, 정수 색인으로 선택하려는 것인지 모호함

In [98]:
ser = pd.Series(np.arange(3), index=[1,2,0])
ser
ser[1] #- 1은 label으로 판단.

0

In [99]:
ser = pd.Series(np.arange(3), index=[1,2,0])

In [100]:
ser

1    0
2    1
0    2
dtype: int64

In [109]:
ser.iloc[2] #- 위치인지 라벨인지 헷갈림. 밑에 ser.iloc[-1] 과 비교

2

In [103]:
#- 정수 기반의 색인을 사용하지 않는 경우 모호함은 사라짐
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

In [104]:
#- 일관성을 유지하기 위해 정숫값을 담고 있는 축 라벨이 있다면 우선적으로 라벨을 찾아보도록 구현되어 있음 
#- 세밀한 표현을 위해서, 라벨에 대해서는 loc을, 정수 색인에 대해서는 iloc을 사용하자
ser[:1]

1    0
dtype: int64

In [105]:
ser.loc[:1] #- 라벨이름으로 슬라이싱하면 시작점과 끝점을 포함함.

1    0
dtype: int64

In [107]:
ser.iloc[:1]
ser.iloc[-1]

2